## Create a simple forecast from the NDFD which can be used with NWS graphics

In [1]:
import numpy as np
from datetime import datetime, timedelta
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy
from metpy.units import units
import metpy.calc as mpcalc
from pyproj import Proj
import pytz
from pytz import timezone
from PIL import Image

### Get current time rounded down to last 30 minute interval

In [2]:
def rounded_to_the_last_30_minute():
    now = datetime.now()
    rounded = now - (now - datetime.min) % timedelta(hours=1)
    return rounded

In [3]:
date = rounded_to_the_last_30_minute()

### Import NDFD data for latest time

In [4]:
YYYYMMDD_HHMM = date.strftime('%Y%m%d_%H%M')

In [5]:
File = "https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_"+YYYYMMDD_HHMM+".grib2"
File

'https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_20240513_0100.grib2'

In [6]:
ds = xr.open_dataset(File)

### Parse for data map projection and add lats & lons

In [7]:
ds = ds.metpy.parse_cf()
ds = ds.metpy.assign_latitude_longitude(force=False)
ds

<xarray.Dataset>
Dimensions:                                                                   (
                                                                               time: 6,
                                                                               time_bounds_1: 2,
                                                                               time2: 12,
                                                                               time2_bounds_1: 2,
                                                                               time3: 3,
                                                                               ...
                                                                               time4_bounds_1: 2,
                                                                               time1: 48,
                                                                               height_above_ground1: 1,
                                                                               x: 2145,
                                                                               y: 1377,
                                                                               height_above_ground: 1)
Coordinates: (12/13)
    reftime                                                                   datetime64[ns] ...
  * time                                                                      (time) datetime64[ns] ...
  * time2                                                                     (time2) datetime64[ns] ...
  * time3                                                                     (time3) datetime64[ns] ...
  * time4                                                                     (time4) datetime64[ns] ...
  * time1                                                                     (time1) datetime64[ns] ...
    ...                                                                        ...
  * x                                                                         (x) float32 ...
  * y                                                                         (y) float32 ...
    metpy_crs                                                                 object ...
  * height_above_ground                                                       (height_above_ground) float32 ...
    latitude                                                                  (y, x) float64 ...
    longitude                                                                 (y, x) float64 ...
Dimensions without coordinates: time_bounds_1, time2_bounds_1, time3_bounds_1,
                                time4_bounds_1
Data variables: (12/20)
    LambertConformal_Projection                                               int32 ...
    time_bounds                                                               (time, time_bounds_1) datetime64[ns] ...
    time2_bounds                                                              (time2, time2_bounds_1) datetime64[ns] ...
    time3_bounds                                                              (time3, time3_bounds_1) datetime64[ns] ...
    time4_bounds                                                              (time4, time4_bounds_1) datetime64[ns] ...
    Dewpoint_temperature_height_above_ground                                  (time1, height_above_ground1, y, x) float32 ...
    ...                                                                        ...
    Total_snowfall_surface_6_Hour_Accumulation                                (time2, y, x) float32 ...
    Apparent_Temperature_height_above_ground                                  (time1, height_above_ground1, y, x) float32 ...
    Weather_string_surface                                                    (time1, y, x) float32 ...
    Wind_direction_from_which_blowing_height_above_ground                     (time1, height_above_ground, y, x) float32 ...
    Wind_speed_height_above_ground                                            (time1, heigh

In [8]:
x, y = ds.x, ds.y

### Define max temp variable & function to get closest gridpoint to ETEC

In [9]:
def find_closest(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx

### Get gridpoint closest to ETEC

In [10]:
proj_data = ds.Temperature_height_above_ground.metpy.cartopy_crs
proj_data;

pFull = Proj(proj_data)

In [11]:
siteName = "ETEC"
siteLat, siteLon = (42.75, -73.80) #lat & lon of gridpoint over ETEC
siteX, siteY = pFull(siteLon, siteLat)
siteXidx, siteYidx = find_closest(x, siteX), find_closest(y, siteY)

In [12]:
ds = ds.isel(x = siteXidx, y = siteYidx).isel()
ds

<xarray.Dataset>
Dimensions:                                                                   (
                                                                               time: 6,
                                                                               time_bounds_1: 2,
                                                                               time2: 12,
                                                                               time2_bounds_1: 2,
                                                                               time3: 3,
                                                                               time3_bounds_1: 2,
                                                                               time4: 3,
                                                                               time4_bounds_1: 2,
                                                                               time1: 48,
                                                                               height_above_ground1: 1,
                                                                               height_above_ground: 1)
Coordinates: (12/13)
    reftime                                                                   datetime64[ns] ...
  * time                                                                      (time) datetime64[ns] ...
  * time2                                                                     (time2) datetime64[ns] ...
  * time3                                                                     (time3) datetime64[ns] ...
  * time4                                                                     (time4) datetime64[ns] ...
  * time1                                                                     (time1) datetime64[ns] ...
    ...                                                                        ...
    x                                                                         float32 ...
    y                                                                         float32 ...
    metpy_crs                                                                 object ...
  * height_above_ground                                                       (height_above_ground) float32 ...
    latitude                                                                  float64 ...
    longitude                                                                 float64 ...
Dimensions without coordinates: time_bounds_1, time2_bounds_1, time3_bounds_1,
                                time4_bounds_1
Data variables: (12/20)
    LambertConformal_Projection                                               int32 ...
    time_bounds                                                               (time, time_bounds_1) datetime64[ns] ...
    time2_bounds                                                              (time2, time2_bounds_1) datetime64[ns] ...
    time3_bounds                                                              (time3, time3_bounds_1) datetime64[ns] ...
    time4_bounds                                                              (time4, time4_bounds_1) datetime64[ns] ...
    Dewpoint_temperature_height_above_ground                                  (time1, height_above_ground1) float32 ...
    ...                                                                        ...
    Total_snowfall_surface_6_Hour_Accumulation                                (time2) float32 ...
    Apparent_Temperature_height_above_ground                                  (time1, height_above_ground1) float32 ...
    Weather_string_surface                                                    (time1) float32 ...
    Wind_direction_from_which_blowing_height_above_ground                     (time1, height_above_ground) float32 ...
    Wind_speed_height_above_ground                                            (time1, height_above_ground) float32 ...
    Wind_speed_gust_height_above_ground                                       (time1, height_abo

In [13]:
temp = ds.Temperature_height_above_ground
timeDim, vertDim = temp.metpy.time.name, temp.metpy.vertical.name
idxVert = 0 # First (and in this case, only) vertical level
idxTime = slice(None, 24)
vertDict = {vertDim: idxVert}
timeDict = {timeDim: idxTime}
temp = temp.isel(vertDict)
temp = temp.isel(timeDict)
temp = temp.drop_vars(['reftime', 'x', 'y', 'metpy_crs', 'longitude', 'latitude', vertDim])
df_temp = temp.to_dataframe()

In [14]:
temp

<xarray.DataArray 'Temperature_height_above_ground' (time1: 24)>
[24 values with dtype=float32]
Coordinates:
  * time1    (time1) datetime64[ns] 2024-05-13T02:00:00 ... 2024-05-14T01:00:00
Attributes: (12/13)
    long_name:                       Temperature @ Specified height level abo...
    units:                           K
    description:                     Temperature
    grid_mapping:                    LambertConformal_Projection
    Grib_Variable_Id:                VAR_0-0-0_L103
    Grib2_Parameter:                 [0 0 0]
    ...                              ...
    Grib2_Parameter_Category:        Temperature
    Grib2_Parameter_Name:            Temperature
    Grib2_Level_Type:                103
    Grib2_Level_Desc:                Specified height level above ground
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  UnknownStatType--1

In [15]:
dewp = ds.Dewpoint_temperature_height_above_ground
dewp = dewp.isel(vertDict)
dewp = dewp.isel(timeDict)
dewp = dewp.drop_vars(['reftime', 'x', 'y', 'metpy_crs', 'longitude', 'latitude', vertDim])
df_dewp = dewp.to_dataframe()

In [16]:
dewp

<xarray.DataArray 'Dewpoint_temperature_height_above_ground' (time1: 24)>
[24 values with dtype=float32]
Coordinates:
  * time1    (time1) datetime64[ns] 2024-05-13T02:00:00 ... 2024-05-14T01:00:00
Attributes: (12/13)
    long_name:                       Dewpoint temperature @ Specified height ...
    units:                           K
    description:                     Dewpoint temperature
    grid_mapping:                    LambertConformal_Projection
    Grib_Variable_Id:                VAR_0-0-6_L103
    Grib2_Parameter:                 [0 0 6]
    ...                              ...
    Grib2_Parameter_Category:        Temperature
    Grib2_Parameter_Name:            Dewpoint temperature
    Grib2_Level_Type:                103
    Grib2_Level_Desc:                Specified height level above ground
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  UnknownStatType--1

In [17]:
wdsp = ds.Wind_speed_height_above_ground
vertDimWind = wdsp.metpy.vertical.name
vertDictWind = {vertDimWind: idxVert}
wdsp = wdsp.isel(vertDictWind)
wdsp = wdsp.isel(timeDict)
wdsp = wdsp.drop_vars(['reftime', 'x', 'y', 'metpy_crs', 'longitude', 'latitude', vertDimWind])
wdsp = wdsp * 2.23694
df_wdsp = wdsp.to_dataframe()

In [18]:
wdsp

<xarray.DataArray 'Wind_speed_height_above_ground' (time1: 24)>
array([ 4.6975737,  4.6975737,  3.3554099,  3.3554099,  3.3554099,
        3.3554099,  3.3554099,  3.3554099,  4.6975737,  4.6975737,
        5.8160434,  5.8160434,  6.9345136,  8.052983 ,  9.171453 ,
        9.171453 ,  9.171453 , 11.408393 , 11.408393 , 11.408393 ,
       11.408393 , 11.408393 , 11.408393 , 10.289924 ], dtype=float32)
Coordinates:
  * time1    (time1) datetime64[ns] 2024-05-13T02:00:00 ... 2024-05-14T01:00:00

In [19]:
df_merge1 = pd.merge(df_temp, df_dewp, on=timeDim)
df = pd.merge(df_merge1, df_wdsp, on=timeDim)
df

,Temperature_height_above_ground,Dewpoint_temperature_height_above_ground,Wind_speed_height_above_ground
time1,,,
2024-05-13 02:00:00,284.799988,279.799988,4.697574
2024-05-13 03:00:00,283.700012,280.399994,4.697574
2024-05-13 04:00:00,283.100006,280.399994,3.355410
2024-05-13 05:00:00,282.600006,279.799988,3.355410
2024-05-13 06:00:00,282.000000,279.799988,3.355410
2024-05-13 07:00:00,280.899994,279.299988,3.355410
2024-05-13 08:00:00,279.799988,278.700012,3.355410
2024-05-13 09:00:00,279.799988,278.700012,3.355410
2024-05-13 10:00:00,280.399994,278.700012,4.697574


In [20]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, AutoDateLocator, YearLocator, HourLocator, DayLocator, MonthLocator

from netCDF4 import num2date

from metpy.units import units
from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS
from datetime import datetime, timedelta

import pandas as pd
import xarray as xr
import metpy
import metpy.calc as mpcalc
from PIL import Image
import pytz
from pytz import timezone

In [21]:
# Albany version is GEMPAK converted to netCDF.
# Two possibilities:  one is the one-year archive, updated once per day; the other is the most-recent week archive, updated in real time.
#metar_cat_url = 'http://thredds.atmos.albany.edu:8080/thredds/catalog/metarArchive/ncdecoded/catalog.xml?dataset=metarArchive/ncdecoded/Archived_Metar_Station_Data_fc.cdmr'
metar_cat_url = 'http://thredds.atmos.albany.edu:8080/thredds/catalog/metar/ncdecoded/catalog.xml?dataset=metar/ncdecoded/Metar_Station_Data_fc.cdmr'
# Parse the xml and return a THREDDS Catalog Object.
catalog = TDSCatalog(metar_cat_url)

metar_dataset = catalog.datasets['Feature Collection']

In [22]:
ncss_url = metar_dataset.access_urls['NetcdfSubset']

In [23]:
# We have the URL for our catalog's NetCDF Subset service, now create an object using the ncss client and pull
ncss = NCSS(ncss_url)

In [24]:
ncss.variables.remove('_isMissing')

In [25]:
# get current date and time

now = datetime.utcnow()
now = datetime(now.year, now.month, now.day, now.hour)
day_1 = now - timedelta(hours = 23, minutes = 30)

# build the query
query = ncss.query()

In [26]:
# Select a location or list of locatons. 
#This can be either a single point (THREDDS will attempt to locate the nearest station) or an actual METAR site ID.

query.add_query_parameter(stns='ALB',subset='stns')

query.time_range(day_1, now)

#query.variables('all')
query.variables('PMSL', 'TMPC', 'DWPC', 'WNUM',
                'DRCT', 'SKNT', 'GUST', 'ALTI', 'CHC1', 'CHC2', 'CHC3')
query.accept('netcdf')

var=SKNT&var=CHC2&var=DWPC&var=PMSL&var=GUST&var=CHC1&var=DRCT&var=CHC3&var=TMPC&var=ALTI&var=WNUM&time_start=2024-05-12T01%3A30%3A00&time_end=2024-05-13T01%3A00%3A00&stns=ALB&subset=stns&accept=netcdf

In [27]:
data = ncss.get_data(query)

In [28]:
data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.6
    history: Written by CFPointWriter
    title: Extracted data from TDS Feature Collection Metar Station Data
    time_coverage_start: 2024-05-12T02:00:00Z
    time_coverage_end: 2024-05-13T00:00:00Z
    geospatial_lat_min: 42.7495
    geospatial_lat_max: 42.7505
    geospatial_lon_min: -73.80050305175781
    geospatial_lon_max: -73.79950305175781
    featureType: timeSeries
    dimensions(sizes): obs(23), station(1), station_id_strlen(3)
    variables(dimensions): float64 latitude(station), float64 longitude(station), float64 stationAltitude(station), |S1 station_id(station, station_id_strlen), float64 time(obs), int32 stationIndex(obs), float32 PMSL(obs), float32 ALTI(obs), float32 TMPC(obs), float32 DWPC(obs), float32 SKNT(obs), float32 DRCT(obs), float32 GUST(obs), float32 WNUM(obs), float32 CHC1(obs), float32 CHC2(obs), float32 CHC3(obs)
    groups: 

In [29]:
station_id = data['station_id'][0].tobytes() #get station id
station_id = station_id.decode('ascii')
print(station_id)

ALB


In [30]:
time_var = data.variables['time'] #get the date & time of metar
#print (time_var)
time = num2date(time_var, time_var.units, only_use_cftime_datetimes=False, only_use_python_datetimes=True)
time

masked_array(data=[real_datetime(2024, 5, 12, 2, 0),
                   real_datetime(2024, 5, 12, 3, 0),
                   real_datetime(2024, 5, 12, 4, 0),
                   real_datetime(2024, 5, 12, 4, 40),
                   real_datetime(2024, 5, 12, 6, 0),
                   real_datetime(2024, 5, 12, 7, 0),
                   real_datetime(2024, 5, 12, 8, 0),
                   real_datetime(2024, 5, 12, 9, 0),
                   real_datetime(2024, 5, 12, 10, 0),
                   real_datetime(2024, 5, 12, 11, 0),
                   real_datetime(2024, 5, 12, 12, 0),
                   real_datetime(2024, 5, 12, 13, 0),
                   real_datetime(2024, 5, 12, 13, 40),
                   real_datetime(2024, 5, 12, 14, 40),
                   real_datetime(2024, 5, 12, 15, 40),
                   real_datetime(2024, 5, 12, 17, 0),
                   real_datetime(2024, 5, 12, 18, 0),
                   real_datetime(2024, 5, 12, 19, 0),
                   real_datetime

In [31]:
tmpc = data.variables['TMPC'] #define variables
dwpc = data.variables['DWPC']
#slp = data.variables['PMSL']
wdsp = data.variables['SKNT']
#wdir = data.variables['DRCT']
#gust = data.variables['GUST']
#pres = data.variables['ALTI']

In [32]:
length = len(time)
hours = np.arange(0, length, 1)

In [33]:
tmpcs = []
dwpcs = []
wdsps = []
i = 0
for x in hours:
    tmpcs.append(tmpc[i].data) 
    dwpcs.append(dwpc[i].data)
    wdsps.append(wdsp[i].data)
    i = i + 1

In [34]:
tmpCs = tmpcs * units('degC') #attch units where necessary
tmpKs = tmpCs.to('K').magnitude

dwpCs = dwpcs * units('degC')
dwpKs = dwpCs.to('K').magnitude

wdsKt = wdsps * units('kt')
wdmph = wdsKt.to('mph').magnitude

In [35]:
df2 = pd.DataFrame(
    { timeDim : time,
     'Temperature_height_above_ground' : tmpKs,
     'Dewpoint_temperature_height_above_ground' : dwpKs,
     'Wind_speed_height_above_ground' : wdmph}   
)

In [36]:
df2 = df2.set_index(df2.columns[0])

In [37]:
df2

,Temperature_height_above_ground,Dewpoint_temperature_height_above_ground,Wind_speed_height_above_ground
time1,,,
2024-05-12 02:00:00,288.149994,277.549988,11.507795
2024-05-12 03:00:00,287.549988,277.549988,10.357016
2024-05-12 04:00:00,286.449982,277.549988,10.357016
2024-05-12 04:40:00,285.350006,278.750000,10.357016
2024-05-12 06:00:00,284.250000,280.350006,9.206236
2024-05-12 07:00:00,283.750000,279.250000,13.809354
2024-05-12 08:00:00,283.149994,278.750000,11.507795
2024-05-12 09:00:00,283.149994,278.750000,9.206236
2024-05-12 10:00:00,282.549988,279.250000,10.357016


In [38]:
df3 = pd.concat([df2, df])
df3 = df3.rename(columns={timeDim: 'Time',
                          "Temperature_height_above_ground": "T",
                          "Dewpoint_temperature_height_above_ground": "Td",
                          "Wind_speed_height_above_ground": "Wind"})

In [39]:
df3 = df3.reset_index()

In [40]:
df3['Time'] = pd.to_datetime(df3[timeDim])

In [41]:
df3['T'] = (df3['T'] - 273.15) * (9/5) + 32
df3['Td'] = (df3['Td'] - 273.15) * (9/5) + 32

In [42]:
tempsF = df3['T']
dewpsF = df3['Td']
windsMph = df3['Wind']

In [43]:
rh = []
i = 0
for temp in tempsF:
    relH = 100 * mpcalc.relative_humidity_from_dewpoint(tempsF[i] * units.degF, dewpsF[i] * units.degF).magnitude
    rh.append(relH)
    i = i + 1

In [44]:
windChills = []
i = 0
for wind in windsMph:
    if tempsF[i] <= 50 and windsMph[i] > 3:
        windChill = 35.74 + (0.6215 * tempsF[i]) - (35.75 * (windsMph[i]**0.16)) + (0.4275 * tempsF[i] * (windsMph[i]**0.16))
        windChills.append(windChill)
    else:
        windChills.append(float('NaN'))
    i = i + 1

In [45]:
heatIndex = []
i = 0
for temp in tempsF:
    if tempsF[i] >= 80:
        hi = -42.379 + 2.04901523*temp + 10.14333127*rh[i] - .22475541*temp*rh[i] - .00683783*temp*temp - .05481717*rh[i]*rh[i] + .00122874*temp*temp*rh[i] + .00085282*temp*rh[i]*rh[i] - .00000199*temp*temp*rh[i]*rh[i]
        if rh[i] < 13:
            adj = (13 - rh[i])/4 * np.sqrt(17 - abs(temp - 95)/17)
            hi = hi - adj
        elif rh[i] > 85 and temp < 87:
            (rh[i] - 85)/10 * (87 - temp)/5
        heatIndex.append(hi)
    else:
        heatIndex.append(float('NaN'))
    i = i + 1

In [46]:
df4 = pd.DataFrame(
    {'WindChill' : windChills}
)

In [47]:
df5 = pd.concat([df3, df4], axis=1)
df5

,time1,T,Td,Wind,Time,WindChill
0,2024-05-12 02:00:00,59.000000,39.919991,11.507795,2024-05-12 02:00:00,NaN
1,2024-05-12 03:00:00,57.919991,39.919991,10.357016,2024-05-12 03:00:00,NaN
2,2024-05-12 04:00:00,55.939980,39.919991,10.357016,2024-05-12 04:00:00,NaN
3,2024-05-12 04:40:00,53.960022,42.080009,10.357016,2024-05-12 04:40:00,NaN
4,2024-05-12 06:00:00,51.980011,44.960022,9.206236,2024-05-12 06:00:00,NaN
5,2024-05-12 07:00:00,51.080009,42.980011,13.809354,2024-05-12 07:00:00,NaN
6,2024-05-12 08:00:00,50.000000,42.080009,11.507795,2024-05-12 08:00:00,45.564625
7,2024-05-12 09:00:00,50.000000,42.080009,9.206236,2024-05-12 09:00:00,46.309943
8,2024-05-12 10:00:00,48.919991,42.980011,10.357016,2024-05-12 10:00:00,44.577507
9,2024-05-12 11:00:00,48.919991,44.060020,6.904677,2024-05-12 11:00:00,45.932189


In [48]:
df6 = pd.DataFrame(
    {'RH' : rh}
)

In [49]:
df7 = pd.concat([df5, df6], axis=1)
df7

,time1,T,Td,Wind,Time,WindChill,RH
0,2024-05-12 02:00:00,59.000000,39.919991,11.507795,2024-05-12 02:00:00,NaN,49.080303
1,2024-05-12 03:00:00,57.919991,39.919991,10.357016,2024-05-12 03:00:00,NaN,51.018160
2,2024-05-12 04:00:00,55.939980,39.919991,10.357016,2024-05-12 04:00:00,NaN,54.797532
3,2024-05-12 04:40:00,53.960022,42.080009,10.357016,2024-05-12 04:40:00,NaN,64.028858
4,2024-05-12 06:00:00,51.980011,44.960022,9.206236,2024-05-12 06:00:00,NaN,76.882059
5,2024-05-12 07:00:00,51.080009,42.980011,13.809354,2024-05-12 07:00:00,NaN,73.691684
6,2024-05-12 08:00:00,50.000000,42.080009,11.507795,2024-05-12 08:00:00,45.564625,74.096418
7,2024-05-12 09:00:00,50.000000,42.080009,9.206236,2024-05-12 09:00:00,46.309943,74.096418
8,2024-05-12 10:00:00,48.919991,42.980011,10.357016,2024-05-12 10:00:00,44.577507,79.856915
9,2024-05-12 11:00:00,48.919991,44.060020,6.904677,2024-05-12 11:00:00,45.932189,83.227294


In [50]:
df8 = pd.DataFrame(
    {'HeatIndex' : heatIndex}
)

In [51]:
df9 = pd.concat([df7, df8], axis=1)
df9

,time1,T,Td,Wind,Time,WindChill,RH,HeatIndex
0,2024-05-12 02:00:00,59.000000,39.919991,11.507795,2024-05-12 02:00:00,NaN,49.080303,NaN
1,2024-05-12 03:00:00,57.919991,39.919991,10.357016,2024-05-12 03:00:00,NaN,51.018160,NaN
2,2024-05-12 04:00:00,55.939980,39.919991,10.357016,2024-05-12 04:00:00,NaN,54.797532,NaN
3,2024-05-12 04:40:00,53.960022,42.080009,10.357016,2024-05-12 04:40:00,NaN,64.028858,NaN
4,2024-05-12 06:00:00,51.980011,44.960022,9.206236,2024-05-12 06:00:00,NaN,76.882059,NaN
5,2024-05-12 07:00:00,51.080009,42.980011,13.809354,2024-05-12 07:00:00,NaN,73.691684,NaN
6,2024-05-12 08:00:00,50.000000,42.080009,11.507795,2024-05-12 08:00:00,45.564625,74.096418,NaN
7,2024-05-12 09:00:00,50.000000,42.080009,9.206236,2024-05-12 09:00:00,46.309943,74.096418,NaN
8,2024-05-12 10:00:00,48.919991,42.980011,10.357016,2024-05-12 10:00:00,44.577507,79.856915,NaN
9,2024-05-12 11:00:00,48.919991,44.060020,6.904677,2024-05-12 11:00:00,45.932189,83.227294,NaN


In [52]:
df9['T'] = round(df9['T'])
df9['Td'] = round(df9['Td'])
df9['Wind'] = round(df9['Wind'])
df9['WindChill'] = round(df9['WindChill'])
df9['RH'] = round(df9['RH'])
df9['HeatIndex'] = round(df9['HeatIndex'])

In [53]:
df9 = df9.drop(columns=timeDim)

In [54]:
df9

,T,Td,Wind,Time,WindChill,RH,HeatIndex
0,59.0,40.0,12.0,2024-05-12 02:00:00,NaN,49.0,NaN
1,58.0,40.0,10.0,2024-05-12 03:00:00,NaN,51.0,NaN
2,56.0,40.0,10.0,2024-05-12 04:00:00,NaN,55.0,NaN
3,54.0,42.0,10.0,2024-05-12 04:40:00,NaN,64.0,NaN
4,52.0,45.0,9.0,2024-05-12 06:00:00,NaN,77.0,NaN
5,51.0,43.0,14.0,2024-05-12 07:00:00,NaN,74.0,NaN
6,50.0,42.0,12.0,2024-05-12 08:00:00,46.0,74.0,NaN
7,50.0,42.0,9.0,2024-05-12 09:00:00,46.0,74.0,NaN
8,49.0,43.0,10.0,2024-05-12 10:00:00,45.0,80.0,NaN
9,49.0,44.0,7.0,2024-05-12 11:00:00,46.0,83.0,NaN


In [55]:
df9.to_csv('ALB_obs_fore.csv')